In [ ]:
# %%
###### Start of import packages ######
import sys
import matplotlib as mpl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from statistics import mean, stdev
from sklearn import linear_model
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import ensemble
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from matplotlib import cm as cm
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import xgboost as xgb


pd.set_option('display.max_rows', 100)
###### To ignore warnings ######
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

###### End of import packages ######

sns.set()

In [ ]:
# %%
dataset = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
dataset['SalePrice'].describe()

In [ ]:
sns.histplot(x='SalePrice', data=dataset, kde=True)
plt.show()

In [ ]:
print(dataset['SalePrice'].skew())
print(dataset['SalePrice'].kurtosis())


In [ ]:
# %%
pd.set_option('display.max_rows', 100)
dataset.isnull().sum()

In [ ]:
# # Inspect Correlation
# # corr = dataset.corrwith(dataset['GarageArea']).to_frame()
# corr = dataset.corr()
# plt.figure(figsize=(50, 50))
# sns.heatmap(corr, cmap='coolwarm', annot=True, fmt='.2f')
# plt.show()

In [ ]:
# # # Category description graphs

# dataset_Category = dataset.select_dtypes(include = object)
# dataset_Category

# def chunks(l, n):
#     return [l[i:i + n] for i in range(0, len(l), n)]


# cols = dataset_Category.columns
# for x in chunks(cols, 5):
#     sns.pairplot(dataset_Category, y_vars=x, x_vars=x)


In [ ]:
# # Numerical description graphs
# dataset_numerical = dataset.select_dtypes(include = ['float64', 'int64'])
# dataset_numerical.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
# %%
# Remove Nulls
# dataset.dropna(axis=1,inplace=True)
dataset.drop(['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC',
                 'Fence', 'MiscFeature','GarageType','GarageFinish'], axis=1, inplace=True)
# for x in range(0, dataset.shape[1]):
#     col = dataset.columns[x]
#     if (dataset[col].dtype == object):
#         dataset[col] = dataset[col].fillna(dataset[col].mode()[0])
#     else:
#         dataset[col] = dataset[col].fillna(dataset[col].mean())



for i, column in enumerate(dataset):
    if (dataset[column].dtype == object):
        dataset[column] = dataset[column].fillna(dataset[column].mode()[0])
    else:
        dataset[column] = dataset[column].fillna(dataset[column].mean())


In [ ]:
# Categorical Data Encoding
# Categorical features
cat_features = np.array([i for i in dataset.columns.tolist() if dataset[i].dtype == 'object'])
enc_list = {}

for i in cat_features:
    enc_list[i] = preprocessing.LabelEncoder()
    dataset[i] = enc_list[i].fit_transform(dataset[i])

In [ ]:

corr = dataset.corrwith(dataset['Exterior2nd']).to_frame()
# corr = dataset['GarageArea'].corr(dataset)
# corr
plt.figure(figsize=(2, 15))
sns.heatmap(corr, cmap='coolwarm', annot=True, fmt='.2f')
plt.show()

In [ ]:
# %%
# Drop Highly Correlated
dataset.drop(['GarageCars', 'TotRmsAbvGrd', '1stFlrSF', 'Exterior2nd',
                 'GarageYrBlt', 'MSSubClass'], axis=1, inplace=True)


In [ ]:
# Drop Low Correlated
drops = ["Street", "LandContour", "Utilities", "LotConfig", "LandSlope",
             "Condition2", "MasVnrType", "BsmtCond", "BsmtFinType2", "BsmtFinSF2",
             "BsmtHalfBath", "LowQualFinSF", "3SsnPorch", "MiscVal", "MoSold",
             "YrSold"]
dataset = dataset.drop(drops, axis=1)


In [ ]:
# # # Numerical description graphs
# dataset_numerical = dataset.select_dtypes(include = ['float64', 'int64'])
# dataset_numerical.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
# dataset['LotArea'].skew()

In [ ]:
# dataset['LotArea'].hist(figsize=(8, 8),  xlabelsize=8, ylabelsize=8)

In [ ]:
# dataset = dataset[(dataset['LotArea'] < 50000)]

In [ ]:
# dataset['LotArea'].hist(figsize=(8, 8),  xlabelsize=8, ylabelsize=8)


In [ ]:
# dataset['LotArea'].skew()

In [ ]:
dataset

In [ ]:
# Drop outliers
dataset = dataset[(dataset['LotArea'] < 50000)]
dataset = dataset[(dataset['MasVnrArea'] < 500)]
dataset = dataset[(dataset['BsmtFinSF1'] < 2300)]
dataset = dataset[(dataset['TotalBsmtSF'] < 5000)]
dataset = dataset[(dataset['GrLivArea'] < 4000)]
dataset = dataset[(dataset['OpenPorchSF'] < 200)]


In [ ]:
# # # Numerical description graphs
dataset_numerical = dataset.select_dtypes(include = ['float64', 'int64'])
dataset_numerical.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
# Drop Not Important
drops = ["RoofMatl", "Heating", "Electrical", "Functional",
             "GarageQual", "GarageCond", "PavedDrive",
             "PoolArea"]
dataset = dataset.drop(drops, axis=1)


In [ ]:
# Drop One-Category
drops = [
'Condition1'  ,
'BldgType'    ,
'ExterCond'    ,
'BsmtQual'     ,
'CentralAir'   ,
'SaleType'     ,
'SaleCondition']
dataset = dataset.drop(drops, axis=1)

In [ ]:
# # Numerical decription graphs
# dataset_numerical = dataset.select_dtypes(include = ['float64', 'int64'])
# dataset_numerical.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
# # Inspect Correlation after removal
# corr = dataset.corr()
# plt.figure(figsize=(50, 50))
# sns.heatmap(corr, cmap='coolwarm', annot=True, fmt='.2f')
# plt.show()

In [ ]:
dataset

In [ ]:
# corr = dataset.corrwith(dataset['SalePrice']).to_frame()
# # corr = dataset['GarageArea'].corr(dataset)
# # corr
# plt.figure(figsize=(2, 15))
# sns.heatmap(corr, cmap='coolwarm', annot=True, fmt='.2f')
# plt.show()

In [ ]:
# dataset["SaleType_WD"] = dataset['SaleType'].apply(lambda x: 1 if x=='WD' else 0)
# dataset["SaleType_CWD"] = dataset['SaleType'].apply(lambda x: 1 if x=='CWD' else 0)
# dataset["SaleType_VWD"] = dataset['SaleType'].apply(lambda x: 1 if x=='VWD' else 0)
# dataset["SaleType"] = dataset['SaleType'].apply(lambda x: 1 if x=="New" else 0)
# dataset["SaleType_COD"] = dataset['SaleType'].apply(lambda x: 1 if x=='COD' else 0)
# dataset["SaleType_Con"] = dataset['SaleType'].apply(lambda x: 1 if x=='Con' else 0)
# dataset["SaleType_ConLw"] = dataset['SaleType'].apply(lambda x: 1 if x=='ConLw' else 0)
# dataset["SaleType_ConLI"] = dataset['SaleType'].apply(lambda x: 1 if x=='ConLI' else 0)
# dataset["SaleType_ConLD"] = dataset['SaleType'].apply(lambda x: 1 if x=='ConLD' else 0)
# dataset["SaleType_Oth"] = dataset['SaleType'].apply(lambda x: 1 if x=='Oth' else 0)
dataset['EnclosedPorch'] = dataset['EnclosedPorch'].apply(lambda x: 1 if x>0 else 0)
dataset['ScreenPorch'] = dataset['ScreenPorch'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
# corr = dataset.corrwith(dataset['SalePrice']).to_frame()
# # corr = dataset['GarageArea'].corr(dataset)
# # corr
# plt.figure(figsize=(2, 15))
# sns.heatmap(corr, cmap='coolwarm', annot=True, fmt='.2f')
# plt.show()

In [ ]:
# # Inspect Correlation after removal
# corr = dataset.corr()
# plt.figure(figsize=(50, 50))
# sns.heatmap(corr, cmap='coolwarm', annot=True, fmt='.2f')
# plt.show()

In [ ]:
# Categorical features
cat_features = np.array([i for i in dataset.columns.tolist() if dataset[i].dtype == 'object'])

In [ ]:
dataset[cat_features]

In [ ]:
# Category Plotting
# i=1
# for col in cat_features:
#     plt.figure(i,figsize=(10,10))
#     plt.subplot(224)
#     sns.histplot(data=dataset,x=dataset[col])
#     i+=1
# plt.show    

In [ ]:
# # Transform to Object
# for x in range(0, dataset.shape[1]):
#     if (dataset[dataset.columns[x]].dtype == object):
#         le = preprocessing.LabelEncoder()
#         le.fit(dataset[dataset.columns[x]])
#         dataset[dataset.columns[x]] = le.transform(dataset[dataset.columns[x]])

# Categorical Data Encoding
# enc_list = {}

# for i in cat_features:
#     enc_list[i] = preprocessing.LabelEncoder()
#     dataset[i] = enc_list[i].fit_transform(dataset[i])


# dataset = pd.get_dummies(dataset,drop_first=True)


In [ ]:
dataset

In [ ]:
# from sklearn.impute import KNNImputer
# imputer = KNNImputer(n_neighbors=10)
# dataset = imputer.fit_transform(dataset)

In [ ]:
# # # plot_features_to_SalePrice
# def chunks(l, n):
#     return [l[i:i + n] for i in range(0, len(l), n)]

# cols = dataset.columns
# for x in chunks(cols, 5):
#     sns.pairplot(dataset, y_vars=['SalePrice'], x_vars=x)

In [ ]:
# # Correlation after transformation
# corr = dataset.corr()
# plt.figure(figsize=(50, 50))
# sns.heatmap(corr, cmap='coolwarm', annot=True, fmt='.2f')
# plt.show()

In [ ]:
# # Numerical decription graphs
dataset_numerical = dataset.select_dtypes(include = ['float64', 'int64'])
dataset_numerical.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
dataset['GarageArea'].skew()

In [ ]:
# # Scale Data
dataset.LotArea = np.log1p(dataset.LotArea)
dataset.MasVnrArea = np.log1p(dataset.MasVnrArea)
dataset.BsmtFinSF1 = np.log1p(dataset.BsmtFinSF1)
dataset.BsmtUnfSF = np.log1p(dataset.BsmtUnfSF)
dataset.TotalBsmtSF = np.log1p(dataset.TotalBsmtSF)
dataset['2ndFlrSF'] = np.log1p(dataset['2ndFlrSF'])
dataset.GrLivArea = np.log1p(dataset.GrLivArea)
dataset.GarageArea = np.log1p(dataset.GarageArea)
dataset.OpenPorchSF = np.log1p(dataset.OpenPorchSF)
dataset.EnclosedPorch = np.log1p(dataset.EnclosedPorch)
dataset.WoodDeckSF = np.log1p(dataset.WoodDeckSF)

# dataset = np.log1p(dataset.iloc[:,1:dataset.columns.size])

In [ ]:
pd.set_option('display.max_columns',None)
dataset

In [ ]:
# # # Scale Price
dataset.SalePrice = np.log1p(dataset.SalePrice)

In [ ]:
dataset['MasVnrArea'] = dataset['MasVnrArea'].apply(lambda x: 1 if x>0 else 0)
# # # Numerical description graphs
dataset_numerical = dataset.select_dtypes(include = ['float64', 'int64'])
dataset_numerical.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
# # # Numerical decription graphs
# dataset_numerical = dataset.select_dtypes(include = ['float64', 'int64'])
# dataset_numerical.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
# dataset_numerical = dataset.select_dtypes(include = ['float64', 'int64'])
# dataset_numerical.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
# Remove Nulls Test
test.drop(['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC',
                 'Fence', 'MiscFeature','GarageType','GarageFinish'], axis=1, inplace=True)
# for x in range(0, test.shape[1]):
#     col = test.columns[x]
#     if (test[col].dtype == object):
#         test[col] = test[col].fillna(test[col].mode()[0])
#     else:
#         test[col] = test[col].fillna(test[col].mean())
for i, column in enumerate(test):
    if (test[column].dtype == object):
        test[column] = test[column].fillna(test[column].mode()[0])
    else:
        test[column] = test[column].fillna(test[column].mean())

In [ ]:
# Categorical Data Encoding
# Categorical features
cat_features = np.array([i for i in test.columns.tolist() if test[i].dtype == 'object'])

for i in cat_features:
    test[i] = enc_list[i].fit_transform(test[i])

In [ ]:
# Drop Highly Corr Test
test.drop(['GarageCars', 'TotRmsAbvGrd', '1stFlrSF', 'Exterior2nd',
                 'GarageYrBlt', 'MSSubClass'], axis=1, inplace=True)

In [ ]:
# Drop Low Corr Test
drops = ["Street", "LandContour", "Utilities", "LotConfig", "LandSlope",
             "Condition2", "MasVnrType", "BsmtCond", "BsmtFinType2", "BsmtFinSF2",
             "BsmtHalfBath", "LowQualFinSF", "3SsnPorch", "MiscVal", "MoSold",
             "YrSold"]
test = test.drop(drops, axis=1)

In [ ]:
# Not Important Test
drops = ["RoofMatl", "Heating", "Electrical", "Functional",
             "GarageQual", "GarageCond", "PavedDrive",
             "PoolArea"]
test = test.drop(drops, axis=1)

In [ ]:
# Drop One-Category
drops = [
'Condition1'  , 
'BldgType'    ,
'ExterCond'    ,
'BsmtQual'     ,
'CentralAir'   ,
'SaleType'     ,
'SaleCondition']
test = test.drop(drops, axis=1)

In [ ]:
# dataset["SaleType_WD"] = dataset['SaleType'].apply(lambda x: 1 if x=='WD' else 0)
# dataset["SaleType_CWD"] = dataset['SaleType'].apply(lambda x: 1 if x=='CWD' else 0)
# dataset["SaleType_VWD"] = dataset['SaleType'].apply(lambda x: 1 if x=='VWD' else 0)
# test["SaleType"] = test['SaleType'].apply(lambda x: 1 if x=='New' else 0)
# dataset["SaleType_COD"] = dataset['SaleType'].apply(lambda x: 1 if x=='COD' else 0)
# dataset["SaleType_Con"] = dataset['SaleType'].apply(lambda x: 1 if x=='Con' else 0)
# dataset["SaleType_ConLw"] = dataset['SaleType'].apply(lambda x: 1 if x=='ConLw' else 0)
# dataset["SaleType_ConLI"] = dataset['SaleType'].apply(lambda x: 1 if x=='ConLI' else 0)
# dataset["SaleType_ConLD"] = dataset['SaleType'].apply(lambda x: 1 if x=='ConLD' else 0)
# dataset["SaleType_Oth"] = dataset['SaleType'].apply(lambda x: 1 if x=='Oth' else 0)
test['EnclosedPorch'] = test['EnclosedPorch'].apply(lambda x: 1 if x>0 else 0)
test['ScreenPorch'] = test['ScreenPorch'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
# Categorical features
cat_features = np.array([i for i in test.columns.tolist() if test[i].dtype == 'object'])

In [ ]:
test

In [ ]:
# # Transform Object Test
# for x in range(0, test.shape[1]):
#         if (test[test.columns[x]].dtype == object):
#             le = preprocessing.LabelEncoder()
#             le.fit(test[test.columns[x]])
#             test[test.columns[x]] = le.transform(
#                 test[test.columns[x]])
# Categorical Data Encoding test
# for i in cat_features:
#     test[i] = enc_list[i].fit_transform(test[i])


# test = pd.get_dummies(test,drop_first=True)

In [ ]:
# test = imputer.fit_transform(test)

In [ ]:
# # # Scale Data
# # # dataset = np.log1p(dataset)
test.BsmtFinSF1 = np.log1p(test.BsmtFinSF1)
test.BsmtUnfSF = np.log1p(test.BsmtUnfSF)
test.TotalBsmtSF = np.log1p(test.TotalBsmtSF)
test.GarageArea = np.log1p(test.GarageArea)
test.GrLivArea = np.log1p(test.GrLivArea)
test.LotArea = np.log1p(test.LotArea)
test.MasVnrArea = np.log1p(test.MasVnrArea)
test.OpenPorchSF = np.log1p(test.OpenPorchSF)
test.EnclosedPorch = np.log1p(test.EnclosedPorch)
test.WoodDeckSF = np.log1p(test.WoodDeckSF)

test['2ndFlrSF'] = np.log1p(test['2ndFlrSF'])
test['MasVnrArea'] = test['MasVnrArea'].apply(lambda x: 1 if x>0 else 0)


In [ ]:
# cols = dataset.columns.tolist()
# test = test.reindex(columns=cols).fillna(0)

In [ ]:
print(dataset.shape)
print(test.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

array = dataset.values
testarray = test.values
n = dataset.shape[1]
n = n-1

# test_X = testarray[:, 0:n]  # features
X = dataset.iloc[:,0:n]  # features
Y = dataset['SalePrice']  # target

In [ ]:
# XX = dataset.iloc[:, ~dataset.columns.isin(['Id','SalePrice'])]
# YY = dataset[['SalePrice']]

def rmse(y_true,y_pred):
    return 1 - np.sqrt(mean_squared_error(y_true,y_pred))

scorer = make_scorer(rmse,greater_is_better=True)


rfr = RandomForestRegressor()
sfs_range = SFS(estimator=rfr,
                k_features=(6,30),
                forward=True,
                floating=False,
                scoring=scorer,
                cv=0,n_jobs=8)

sfs_range.fit(X,Y)

# print the accuracy of the best combination as well as the set of best features
print('best combination (ACC: %.3f): %s\n' % (sfs_range.k_score_, sfs_range.k_feature_idx_))

plt.rcParams["figure.figsize"] = (6,6)
# use the plot_sfs to visualize all accuracies
plot_sfs(sfs_range.get_metric_dict(), kind='std_err')

In [ ]:
X_sfs = sfs_range.transform(X)
x_train, x_test, y_train, y_test = train_test_split(X_sfs, Y, test_size=0.2, random_state=200)  # test = 20%, train = 80%


In [ ]:
# from sklearn.discriminant_analysis import StandardScaler
# from sklearn.pipeline import Pipeline


# pipeline1 = Pipeline([
#     ('scaler', StandardScaler()),
#     ('linearregression', LinearRegression())
# ])

# param_grid = {'linearregression__fit_intercept': [True, False],
#               'linearregression__copy_X': [True, False]
#               }

# grid_search1 = GridSearchCV(pipeline1, param_grid, cv=12)
# grid_search1.fit(X,Y)

# print('Parameters : ', grid_search1.best_params_,'\nAccuracy Score : ', grid_search1.best_score_)

In [ ]:
# from sklearn.tree import DecisionTreeRegressor

# pipeline2 = Pipeline([
#     ('scaler', StandardScaler()),
#     ('clf2', DecisionTreeRegressor(random_state=42))
# ])

# param_grid2 = {
#     'clf2__max_depth': [2, 3, 4, 5],
#     'clf2__min_samples_split': [2, 5, 10],
#     'clf2__min_samples_leaf': [1, 2, 4],
#     'clf2__max_leaf_nodes': [None, 5, 10, 20]
# }

# grid_search2 = GridSearchCV(pipeline2, param_grid2, cv=15)
# grid_search2.fit(X,Y)

# print('Parameters : ', grid_search2.best_params_,'\nAccuracy Score : ', grid_search2.best_score_)

In [ ]:
# pipeline5 = Pipeline([
#     ('scaler', StandardScaler()),
#     ('clf5', RandomForestRegressor())
# ])

# param_grid5 = {
#     'clf5__n_estimators': [10, 50],
#     'clf5__max_features': ['auto', 'sqrt'],
#     'clf5__max_depth': [5, 15],
#     'clf5__min_samples_split': [2, 5],
#     'clf5__min_samples_leaf': [1, 3],
# }

# grid_search5 = GridSearchCV(pipeline5, param_grid5, cv=10,n_jobs=8)
# grid_search5.fit(X,Y)

# print('Parameters : ', grid_search5.best_params_,'\nAccuracy Score : ', grid_search5.best_score_)

In [ ]:
# print("\nRandom Forest")
# rfr = RandomForestRegressor(max_depth=15,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=100)
# rfr.fit(X, Y)

In [ ]:
# result = np.exp(rfr.predict(test.iloc[:,1:35]))
# output = pd.DataFrame({'Id': test.Id, 'SalePrice': result})
# print(output)
# output.to_csv('submission.csv', index=False)

In [ ]:
# from sklearn.discriminant_analysis import StandardScaler
# from sklearn.pipeline import Pipeline


# pipeline6 = Pipeline([
#     ('scaler', StandardScaler()),
#     ('clf6', ensemble.GradientBoostingRegressor())
# ])

# param_grid6 = {
#     'clf6__n_estimators': [1000,2000,3000,4000],
#     'clf6__random_state': [1000,1234,1269],
#     'clf6__max_features': ['auto', 'sqrt', 'log2'],
#     'clf6__max_depth': [3,5, 15],
#     'clf6__min_samples_split': [2, 5,10,15],
#     'clf6__min_samples_leaf': [1, 3 ,6,11,15],
# }

# grid_search6 = GridSearchCV(pipeline6, param_grid6, cv=5,n_jobs=8)
# grid_search6.fit(x_train,y_train)

# print('Parameters : ', grid_search6.best_params_,'\nAccuracy Score : ', grid_search6.best_score_)

In [ ]:
# print("\nElasticNet")
# ens_test = linear_model.ElasticNetCV(alphas=[0.0001, 0.0005, 0.001, 0.0015, 0.01, 0.015, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1], 
#                                      l1_ratio=[0.01, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99], max_iter=10000).fit(x_train, y_train)
# ens_result = ens_test.predict(x_test)
# print('\tR2: {}'.format(r2_score(ens_result, y_test)))
# print('\tRMSE: {}'.format(np.sqrt(mean_squared_error(ens_result, y_test))))
# scores = cross_val_score(ens_test, X, Y, cv=5)
# print("\tAccuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


In [ ]:
# print("\nDescisionTreeRegressor")
# dtr = DescisionTreeRegressor(2,2,2,'None')
# dtr.fit(x_train, y_train)
# preds = dtr.predict(x_test)
# scores = cross_val_score(dtr, X, Y, cv=5)
# rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE: %f" % (rmse))
# print("\tAccuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# print("\nXgBoost")
# xg_reg = xgb.XGBRegressor(objective='reg:linear', eval_metric='logloss',
#                           scoring='neg_mean_squared_error', subsample=0.95, colsample_bytree=0.3, learning_rate=0.04,
#                           max_depth=2, alpha=0.1, n_estimators=1000,)
# xg_reg.fit(x_train, y_train)
# preds = xg_reg.predict(x_test)
# scores = cross_val_score(xg_reg, X, Y, cv=5)
# rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE: %f" % (rmse))
# print("\tAccuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
print("\nGradientBoostingRegressor")
g_best = ensemble.GradientBoostingRegressor(n_estimators=1850, random_state=1234, learning_rate=0.02, max_depth=3,
                                            max_features='log2', min_samples_leaf=11, min_samples_split=15, loss='huber').fit(x_train, y_train)
g_best_result = g_best.predict(x_test)
print('\tR2: {}'.format(r2_score(g_best_result, y_test)))
print('\tRMSE: {}'.format(np.sqrt(mean_squared_error(g_best_result, y_test))))
scores = cross_val_score(g_best, X, Y, cv=5)
print("\tAccuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# print("\nLinearRegression")
# lr = linear_model.LinearRegression()
# lr.fit(x_train, y_train)
# lr_result = lr.predict(x_test)
# print('\tR2: {}'.format(r2_score(lr_result, y_test)))
# print('\tRMSE: {}'.format(np.sqrt(mean_squared_error(lr_result, y_test))))
# scores = cross_val_score(lr, X, Y, cv=5)
# print("\tAccuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# print("\nRidge Regression")
# ridge = Ridge(max_iter=50000)
# ridge_est = GridSearchCV(
#     ridge, param_grid={"alpha": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]})
# ridge_est.fit(x_train, y_train)
# ridge_result = ridge_est.predict(x_test)
# print('\tR2: {}'.format(r2_score(ridge_result, y_test)))
# print('\tRMSE: {}'.format(np.sqrt(mean_squared_error(ridge_result, y_test))))
# scores = cross_val_score(ridge_est, X, Y, cv=5)
# print("\tAccuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# print("\nLasso Regression")
# lasso = Lasso(max_iter=50000)
# lasso_est = GridSearchCV(
#     lasso, param_grid={"alpha": np.arange(0.0005, 0.001, 0.00001)})
# lasso_est.fit(x_train, y_train)
# lasso_result = lasso_est.predict(x_test)
# print('\tR2: {}'.format(r2_score(lasso_result, y_test)))
# print('\tRMSE: {}'.format(np.sqrt(mean_squared_error(lasso_result, y_test))))
# scores = cross_val_score(lasso_est, X, Y, cv=5)
# print("\tAccuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
test

In [ ]:
dataset

In [ ]:
result = np.exp(g_best.predict(test.iloc[:,[...]]))
output = pd.DataFrame({'Id': test.Id, 'SalePrice': result})
print(output)
output.to_csv('submission.csv', index=False)